In [3]:
from datasets import Dataset
from datetime import datetime, timedelta
import os
import re
import snowflake.connector
from snowflake.connector.pandas_tools import write_pandas
from time import sleep
import torch
from transformers import pipeline

### GLOBAL VARS | GLOBAL VARS | GLOBAL VARS | GLOBAL VARS | GLOBAL VARS | GLOBAL VARS | GLOBAL VARS | GLOBAL VARS | GLOBAL VARS | GLOBAL VARS | GLOBAL VARS 
### GLOBAL VARS | GLOBAL VARS | GLOBAL VARS | GLOBAL VARS | GLOBAL VARS | GLOBAL VARS | GLOBAL VARS | GLOBAL VARS | GLOBAL VARS | GLOBAL VARS | GLOBAL VARS 
### GLOBAL VARS | GLOBAL VARS | GLOBAL VARS | GLOBAL VARS | GLOBAL VARS | GLOBAL VARS | GLOBAL VARS | GLOBAL VARS | GLOBAL VARS | GLOBAL VARS | GLOBAL VARS 

# this basically means "smoke em if you got em" where the "em" is NVIDIA GPU
DEVICE = 0 if torch.cuda.is_available() else -1

SF_USR = os.getenv('SF_USR')
SF_ID  = os.getenv('SF_ID')
SF_WH  = os.getenv('SF_WH')
SF_DB  = os.getenv('SF_DB')
SF_SC  = os.getenv('SF_SC')
SF_RL  = os.getenv('SF_RL')

# connect to database and init a cursor for querying
xct_params = {
    "user":                 SF_USR
   ,"account":              SF_ID
   ,"warehouse":            SF_WH
   ,"database":             SF_DB
   ,"schema":               SF_SC
   ,"role":                 SF_RL
   ,"private_key_file":     os.getenv('PRIVATE_KEY_PATH')
   ,"private_key_file_pwd": os.getenv('PRIVATE_KEY_PASSPHRASE')
   ,"authenticator":        os.getenv('SF_AUTH')
}

SF_XCT = snowflake.connector.connect(**xct_params) #connection object
CSR = SF_XCT.cursor()

In [ ]:
query = f"""select * from {SF_DB}.{SF_SC}.threads_keywords_summarystats""" 
CSR.execute(query)
threadsdf = CSR.fetch_pandas_all()

With NLP posts grouped as threads to minimize pseudoreplication, we could calculate confidence intervals for positive, negative, and neutral sentiment simultaneously. The Sison and Glaz method is designed for a multinomial proportion like this with more than 2 possible outcomes but may not be the best option here.

Chi Squares Goodness-of-fit test

For now, we assume that the null hypothesis-- which would represent a hypothetical social media platform with no political leaning-- is a 33/33/33% split between the three sentiment categories. In the future I hope to integrate real poll data to track approval ratings of keywords. 

In the future we can also add in translations from other languages in the dataset.

In [14]:
##--Chi Squares Test

import pandas as pd
import numpy as np
from scipy.stats import chi2

ALPHA = 0.05
DEGREES_OF_FREEDOM = 2

In [ ]:
#hypothetical perfectly even sentiment split
threadsdf['EXPECTED_THREADS_PER_SENTIMENT'] = threadsdf['TOTAL_THREADS'] / 3

CHI_SQUARED_POSITIVE = ((threadsdf['POSITIVE_THREADS'] - threadsdf['EXPECTED_THREADS_PER_SENTIMENT'])**2) / threadsdf['EXPECTED_THREADS_PER_SENTIMENT']
CHI_SQUARED_NEUTRAL = ((threadsdf['NEUTRAL_THREADS'] - threadsdf['EXPECTED_THREADS_PER_SENTIMENT'])**2) / threadsdf['EXPECTED_THREADS_PER_SENTIMENT']
CHI_SQUARED_NEGATIVE = ((threadsdf['NEGATIVE_THREADS'] - threadsdf['EXPECTED_THREADS_PER_SENTIMENT'])**2) / threadsdf['EXPECTED_THREADS_PER_SENTIMENT']

threadsdf['CHI_SQUARED_STATISTIC'] = CHI_SQUARED_POSITIVE + CHI_SQUARED_NEUTRAL + CHI_SQUARED_NEGATIVE

threadsdf['P_VALUE'] = chi2.sf(threadsdf['CHI_SQUARED_STATISTIC'], DEGREES_OF_FREEDOM)

threadsdf['SIGNIFICANTLY_DIFFERENT_FROM_EVEN_SPLIT'] = threadsdf['P_VALUE'] < ALPHA
print(threadsdf)

not_significantly_different = threadsdf[~threadsdf['SIGNIFICANTLY_DIFFERENT_FROM_EVEN_SPLIT']].copy()
print(not_significantly_different)

Empty DataFrame
Columns: [KEYWORD, TOPIC, CATEGORY, TOTAL_AUTHORS, TOTAL_THREADS, TOTAL_POSTS, TOTAL_POSITIVE_POSTS, TOTAL_NEUTRAL_POSTS, TOTAL_NEGATIVE_POSTS, POSITIVE_THREADS, NEUTRAL_THREADS, NEGATIVE_THREADS, PERCENT_POSITIVE, PERCENT_NEUTRAL, PERCENT_NEGATIVE, AVG_SENTIMENT_CONFIDENCE, EXPECTED_THREADS_PER_SENTIMENT, CHI_SQUARED_STATISTIC, P_VALUE, SIGNIFICANTLY_DIFFERENT_FROM_EVEN_SPLIT]
Index: []
